# Getting Started with Agent2Agent (A2A) Protocol 

## Overview

This notebook guides you through deploying and managing a conversational AI agent to **Vertex AI Agent Engine**. 

## Learning Goals

By the end of this notebook, you will understand how to:
* Set up required configuration and deploy ADK **Agents** to **Vertex AI Agent Engine**.
* Use the ADK **RemoteAgent** ans **RemoteSession** to execute agent interactions.
* Use the Vertex AI SDK to Manage (list/update) and delete agents that you have deployed to Vertex AI Agent Engine.






## A Purchasing Concierge and Remote Seller Agent Interactions on Cloud Run and Agent Engine

## Overview

This notebook guides you through deploying and managing a conversational AI agent to **Vertex AI Agent Engine**. 

## Learning Goals

By the end of this notebook, you will understand how to:
* Set up required configuration and deploy ADK **Agents** to **Vertex AI Agent Engine**.
* Use the ADK **RemoteAgent** ans **RemoteSession** to execute agent interactions.
* Use the Vertex AI SDK to Manage (list/update) and delete agents that you have deployed to Vertex AI Agent Engine.


- Core structure of A2A Server
- Core structure of A2A Client
- Deploying agent service to Cloud Run
- Deploying agent service to Agent Engine
- How A2A Client connect to A2A Server
- Request and Response structure on non-streaming connection



Agent2Agent (A2A) protocol is designed to standardize communication between AI agents, particularly for those which are deployed in external systems. Previously, such protocols were established for Tools called Model Context Protocol (MCP) which is an emerging standard to connect LLMs with data and resources. A2A tries to complement MCP where A2A is focused on a different problem, while MCP focuses on lowering complexity to connect agents with tools and data, A2A focuses on how to enable agents to collaborate in their natural modalities. It allows agents to communicate as agents (or as users) instead of as tools; for example, enable back-and-forth communication when you want to order something.

A2A is positioned to complement MCP, in the official documentation it is recommended that applications use MCP for tools and A2A for agents - represented by AgentCard ( We will discuss this later on ). The frameworks can then use A2A to communicate with their user, the remote agents, and other agents.


## What you'll learn
- Core structure of A2A Server
- Core structure of A2A Client
- Deploying agent service to Cloud Run
- Deploying agent service to Agent Engine
- How A2A Client connect to A2A Server
- Request and Response structure on non-streaming connection

## Setup
This lab needs a special kernel to run, please run the following cell.
**NOTE: You can skip this step if you have already built the ADK Kernel from the previous Lab**

In [ ]:
!echo "Kernel installation started."
!cd ../../.. && make adk_mcp_a2a_kernel > /dev/null 2>&1
!echo "Kernel installation completed."

When it's completed, select the **`ADK MCP A2A Kernel`** on the top right before going forward in the notebook.<br>
It may take ~1 minutes until the kernel is shown after the installation.

## Import Python Packages

In [ ]:
import os
import requests
import json

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
REGION="us-central1"
BUCKET_NAME = f"agent-deployment-a2a-{PROJECT_ID}-bucket"
STAGING_BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
os.environ["REGION"]=REGION
os.environ["PROJECT_ID"]=PROJECT_ID

### Defining an auxiliary magic function

The magic function `writefile` from Jupyter Notebook can only write the cell as is and could not unpack Python variables. Hence, we need to create an auxiliary magic function that can unpack Python variables and write them to a file.

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "a") as f:
        f.write(cell.format(**globals()))
        
@register_line_cell_magic
def writeconfig(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

### 

## Deploying A2A Server Remote Seller Agents to Cloud Run

In this step, we will deploy these two remote seller agents marked by the red box. 
 - The burger agent will be powered by CrewAI agent framework 
 - The pizza agent will be powered by Langgraph agent

### Deploying Burger Seller Agent - A2A Server

#### The burger agent source code is under the remote_seller_agents/burger_agent directory.
All files that exist under remote_seller_agents/burger_agent directory are already sufficient to deploy our agent to Cloud Run so that it can be accessible as a service. 
#### Run the following command to deploy it

In [ ]:
%%bash
gcloud run deploy burger-agent \
    --source remote_seller_agents/burger_agent \
    --port=8080 \
    --allow-unauthenticated \
    --min 1 \
    --max 1 \
    --region us-central1 \
    --update-env-vars GOOGLE_CLOUD_LOCATION=$REGION \
    --update-env-vars GOOGLE_CLOUD_PROJECT=$PROJECT_ID

In [ ]:
# to get the service URL programmatically
SERVICE_URL = !gcloud run services describe burger-agent \
  --platform managed \
  --region $REGION \
  --format "value(status.url)"

BURGER_AGENT_SERVICE_URL = SERVICE_URL[0]
print(BURGER_AGENT_SERVICE_URL)

os.environ["BURGER_AGENT_HOST_OVERRIDE"] = BURGER_AGENT_SERVICE_URL

In [ ]:
%%bash
gcloud run services update burger-agent --region=us-central1 --update-env-vars=HOST_OVERRIDE=$BURGER_AGENT_HOST_OVERRIDE

In [ ]:
## 


The a2a/.well-known/agent.json API is a crucial component of the Agent-to-Agent (A2A) communication protocol and
provides a standardized way for one AI agent (a client) to discover the capabilities of another AI agent (a server). This is part of a broader web standard where the /.well-known/ URL path is used for discovering information about a site or service.
When a client agent wants to interact with a server agent, it first makes a GET request to this specific URL. The server then responds with a JSON file that contains vital information about itself.

In [ ]:
res = requests.get(f'{BURGER_AGENT_SERVICE_URL}/.well-known/agent.json')
print(json.dumps(json.loads(res.content), indent=4))

In [ ]:
%%bash
gcloud run deploy pizza-agent \
    --source remote_seller_agents/pizza_agent \
    --port=8080 \
    --allow-unauthenticated \
    --min 1 \
    --max 1 \
    --region us-central1 \
    --update-env-vars GOOGLE_CLOUD_LOCATION=$REGION \
    --update-env-vars GOOGLE_CLOUD_PROJECT=$PROJECT_ID

In [ ]:
# to get the service URL programmatically
PIZZA_AGENT_SERVICE_URL = !gcloud run services describe pizza-agent \
  --platform managed \
  --region $REGION \
  --format "value(status.url)"

PIZZA_AGENT_SERVICE_URL = PIZZA_AGENT_SERVICE_URL[0]
print(PIZZA_AGENT_SERVICE_URL)

os.environ["PIZZA_AGENT_SERVICE_URL"] = PIZZA_AGENT_SERVICE_URL

In [ ]:
%%bash
gcloud run services update pizza-agent --region=us-central1 --update-env-vars=HOST_OVERRIDE=$PIZZA_AGENT_SERVICE_URL

In [ ]:
res = requests.get(f'{PIZZA_AGENT_SERVICE_URL}/.well-known/agent.json')
print(json.dumps(json.loads(res.content), indent=4))

**Checking for the existence of BUCKET. Creating it if it doesn't exist:**

In [ ]:
!gsutil ls $STAGING_BUCKET_URI || gsutil mb -l $LOCATION $STAGING_BUCKET_URI

In [ ]:
%%writeconfig .env
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

GOOGLE_GENAI_USE_VERTEXAI=TRUE
PIZZA_SELLER_AGENT_URL={PIZZA_AGENT_SERVICE_URL}
BURGER_SELLER_AGENT_URL={BURGER_AGENT_SERVICE_URL}
GOOGLE_CLOUD_PROJECT={PROJECT_ID}
GOOGLE_CLOUD_LOCATION={REGION}
STAGING_BUCKET={STAGING_BUCKET_URI}
AGENT_ENGINE_RESOURCE_NAME=your-agent-engine-resource-name

### Check .env file content

In [ ]:
!cat .env

In [ ]:
ADK_AGENT_PYTHON = './purchasing_concierge/purchasing_agent.py'

In [ ]:
%%writefile {ADK_AGENT_PYTHON}
"""
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import json
import uuid
from typing import List
import httpx

from google.adk import Agent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.tools.tool_context import ToolContext
from .remote_agent_connection import RemoteAgentConnections

from a2a.client import A2ACardResolver
from a2a.types import (
    AgentCard,
    MessageSendParams,
    Part,
    SendMessageRequest,
    SendMessageResponse,
    SendMessageSuccessResponse,
    Task,
)


class PurchasingAgent:
    """The purchasing agent.

    This is the agent responsible for choosing which remote seller agents to send
    tasks to and coordinate their work.
    """

    def __init__(
        self,
        remote_agent_addresses: List[str],
    ):
        self.remote_agent_connections: dict[str, RemoteAgentConnections] = {}
        self.remote_agent_addresses = remote_agent_addresses
        self.cards: dict[str, AgentCard] = {}
        self.agents = ""
        self.a2a_client_init_status = False

    def create_agent(self) -> Agent:
        return Agent(
            model="gemini-2.5-flash",
            name="purchasing_agent",
            instruction=self.root_instruction,
            before_model_callback=self.before_model_callback,
            before_agent_callback=self.before_agent_callback,
            description=(
                "This purchasing agent orchestrates the decomposition of the user purchase request into"
                " tasks that can be performed by the seller agents."
            ),
            tools=[
                self.send_task,
            ],
        )

    def root_instruction(self, context: ReadonlyContext) -> str:
        current_agent = self.check_active_agent(context)
        return f"""You are an expert purchasing delegator that can delegate the user product inquiry and purchase request to the
appropriate seller remote agents.

Execution:
- For actionable tasks, you can use `send_task` to assign tasks to remote agents to perform.
- When the remote agent is repeatedly asking for user confirmation, assume that the remote agent doesn't have access to user's conversation context. 
    So improve the task description to include all the necessary information related to that agent
- Never ask user permission when you want to connect with remote agents. If you need to make connection with multiple remote agents, directly
    connect with them without asking user permission or asking user preference
- Always show the detailed response information from the seller agent and propagate it properly to the user. 
- If the remote seller is asking for confirmation, rely the confirmation question with proper and necessary information to the user if the user haven't do so. 
- If the user already confirmed the related order in the past conversation history, you can confirm on behalf of the user
- Do not give irrelevant context to remote seller agent. For example, ordered pizza item is not relevant for the burger seller agent
- Never ask order confirmation to the remote seller agent 

Please rely on tools to address the request, and don't make up the response. If you are not sure, please ask the user for more details.
Focus on the most recent parts of the conversation primarily.

If there is an active agent, send the request to that agent with the update task tool.

Agents:
{self.agents}

Current active seller agent: {current_agent["active_agent"]}
"""

    def check_active_agent(self, context: ReadonlyContext):
        state = context.state
        if (
            "session_id" in state
            and "session_active" in state
            and state["session_active"]
            and "active_agent" in state
        ):
            return {"active_agent": f"{state['active_agent']}"}
        return {"active_agent": "None"}

    async def before_agent_callback(self, callback_context: CallbackContext):
        if not self.a2a_client_init_status:
            httpx_client = httpx.AsyncClient(timeout=httpx.Timeout(timeout=30))
            for address in self.remote_agent_addresses:
                card_resolver = A2ACardResolver(
                    base_url=address, httpx_client=httpx_client
                )
                try:
                    card = await card_resolver.get_agent_card()
                    remote_connection = RemoteAgentConnections(
                        agent_card=card, agent_url=card.url
                    )
                    self.remote_agent_connections[card.name] = remote_connection
                    self.cards[card.name] = card
                except httpx.ConnectError:
                    print(f"ERROR: Failed to get agent card from : {address}")
            agent_info = []
            for ra in self.list_remote_agents():
                agent_info.append(json.dumps(ra))
            self.agents = "\n".join(agent_info)
            self.a2a_client_init_status = True

    async def before_model_callback(
        self, callback_context: CallbackContext, llm_request
    ):
        state = callback_context.state
        if "session_active" not in state or not state["session_active"]:
            if "session_id" not in state:
                state["session_id"] = str(uuid.uuid4())
            state["session_active"] = True

    def list_remote_agents(self):
        """List the available remote agents you can use to delegate the task."""
        if not self.remote_agent_connections:
            return []

        remote_agent_info = []
        for card in self.cards.values():
            print(f"Found agent card: {card.model_dump()}")
            print("=" * 100)
            remote_agent_info.append(
                {"name": card.name, "description": card.description}
            )
        return remote_agent_info

    def send_task(self, agent_name: str, task: str, tool_context: ToolContext):
        """Sends a task to remote seller agent

        This will send a message to the remote agent named agent_name.

        Args:
            agent_name: The name of the agent to send the task to.
            task: The comprehensive conversation context summary
                and goal to be achieved regarding user inquiry and purchase request.
            tool_context: The tool context this method runs in.

        Yields:
            A dictionary of JSON data.
        """
        if agent_name not in self.remote_agent_connections:
            raise ValueError(f"Agent {agent_name} not found")
        state = tool_context.state
        state["active_agent"] = agent_name
        client = self.remote_agent_connections[agent_name]
        if not client:
            raise ValueError(f"Client not available for {agent_name}")
        session_id = state["session_id"]
        task: Task
        message_id = ""
        metadata = {}
        if "input_message_metadata" in state:
            metadata.update(**state["input_message_metadata"])
            if "message_id" in state["input_message_metadata"]:
                message_id = state["input_message_metadata"]["message_id"]
        if not message_id:
            message_id = str(uuid.uuid4())

        payload = {
            "message": {
                "role": "user",
                "parts": [
                    {"type": "text", "text": task}
                ],  # Use the 'task' argument here
                "messageId": message_id,
                "contextId": session_id,
            },
        }

        message_request = SendMessageRequest(
            id=message_id, params=MessageSendParams.model_validate(payload)
        )
        send_response: SendMessageResponse = client.send_message(
            message_request=message_request
        )
        print(
            "send_response",
            send_response.model_dump_json(exclude_none=True, indent=2),
        )

        if not isinstance(send_response.root, SendMessageSuccessResponse):
            print("received non-success response. Aborting get task ")
            return None

        if not isinstance(send_response.root.result, Task):
            print("received non-task response. Aborting get task ")
            return None

        return send_response.root.result


def convert_parts(parts: list[Part], tool_context: ToolContext):
    rval = []
    for p in parts:
        rval.append(convert_part(p, tool_context))
    return rval


def convert_part(part: Part, tool_context: ToolContext):
    # Currently only support text parts
    if part.type == "text":
        return part.text

    return f"Unknown type: {part.type}"

In [ ]:
AGENT_REMOTE_CONNECTION_PYTHON = './purchasing_concierge/remote_agent_connection.py'

In [ ]:
%%writefile {AGENT_REMOTE_CONNECTION_PYTHON}
from typing import Callable

import httpx

from a2a.client import A2AClient
from a2a.types import (
    AgentCard,
    SendMessageRequest,
    SendMessageResponse,
    Task,
    TaskArtifactUpdateEvent,
    TaskStatusUpdateEvent,
)
from uuid import uuid4
from dotenv import load_dotenv
import json
from typing import Any
from a2a.client.errors import (
    A2AClientHTTPError,
    A2AClientJSONError,
    A2AClientTimeoutError,
)
from a2a.client.middleware import ClientCallContext
import requests

load_dotenv()

TaskCallbackArg = Task | TaskStatusUpdateEvent | TaskArtifactUpdateEvent
TaskUpdateCallback = Callable[[TaskCallbackArg, AgentCard], Task]


def _send_request(
    self,
    rpc_request_payload: dict[str, Any],
    http_kwargs: dict[str, Any] | None = None,
) -> dict[str, Any]:
    """Sends a non-streaming JSON-RPC request to the agent.

    Args:
        rpc_request_payload: JSON RPC payload for sending the request.
        http_kwargs: Optional dictionary of keyword arguments to pass to the
            underlying post request.

    Returns:
        The JSON response payload as a dictionary.

    Raises:
        A2AClientHTTPError: If an HTTP error occurs during the request.
        A2AClientJSONError: If the response body cannot be decoded as JSON.
    """
    try:
        response = requests.post(
            self.url, json=rpc_request_payload, **(http_kwargs or {})
        )
        response.raise_for_status()
        return response.json()
    except httpx.ReadTimeout as e:
        raise A2AClientTimeoutError("Client Request timed out") from e
    except httpx.HTTPStatusError as e:
        raise A2AClientHTTPError(e.response.status_code, str(e)) from e
    except json.JSONDecodeError as e:
        raise A2AClientJSONError(str(e)) from e
    except httpx.RequestError as e:
        raise A2AClientHTTPError(503, f"Network communication error: {e}") from e


def send_message(
    self,
    request: SendMessageRequest,
    *,
    http_kwargs: dict[str, Any] | None = None,
    context: ClientCallContext | None = None,
) -> SendMessageResponse:
    """Sends a non-streaming message request to the agent.

    Args:
        request: The `SendMessageRequest` object containing the message and configuration.
        http_kwargs: Optional dictionary of keyword arguments to pass to the
            underlying httpx.post request.
        context: The client call context.

    Returns:
        A `SendMessageResponse` object containing the agent's response (Task or Message) or an error.

    Raises:
        A2AClientHTTPError: If an HTTP error occurs during the request.
        A2AClientJSONError: If the response body cannot be decoded as JSON or validated.
    """
    if not request.id:
        request.id = str(uuid4())

    response_data = self._send_request(
        request.model_dump(mode="json", exclude_none=True), http_kwargs
    )
    return SendMessageResponse.model_validate(response_data)


class RemoteAgentConnections:
    """A class to hold the connections to the remote agents."""

    def __init__(self, agent_card: AgentCard, agent_url: str):
        print(f"agent_card: {agent_card}")
        print(f"agent_url: {agent_url}")
        self._httpx_client = httpx.AsyncClient(timeout=30)
        self.agent_client = A2AClient(self._httpx_client, agent_card, url=agent_url)

        # Replace the original method with our custom implementation
        # NOTE: This is a temporary workaround for issue in httpx event closed
        self.agent_client._send_request = _send_request.__get__(self.agent_client)
        self.agent_client.send_message = send_message.__get__(self.agent_client)

        self.card = agent_card

    def get_agent(self) -> AgentCard:
        return self.card

    def send_message(self, message_request: SendMessageRequest) -> SendMessageResponse:
        return self.agent_client.send_message(message_request)


In [ ]:
ADK_AGENT_PYTHON = './purchasing_concierge/agent.py'

In [ ]:
%%writefile {ADK_AGENT_PYTHON}
from .purchasing_agent import PurchasingAgent
from dotenv import load_dotenv
import os

load_dotenv(os.path.join(os.path.dirname(__file__), ".env"))

root_agent = PurchasingAgent(
    remote_agent_addresses=[
        os.getenv("PIZZA_SELLER_AGENT_URL", "http://localhost:10000"),
        os.getenv("BURGER_SELLER_AGENT_URL", "http://localhost:10001"),
    ]
).create_agent()

In [ ]:
import vertexai
from vertexai.preview import reasoning_engines
from vertexai import agent_engines
from dotenv import load_dotenv
import os
from purchasing_concierge.agent import root_agent

#importlib.reload(agent)  # Force reload


vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET_URI,
)

adk_app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True
)

remote_app = agent_engines.create(
    agent_engine=adk_app,
    display_name="purchasing-concierge",
    requirements=[
        "google-cloud-aiplatform[agent_engines]==1.115.0",
        "google-adk==1.15.1",
        "a2a-sdk==0.2.16",
    ],
    extra_packages=[
        "./purchasing_concierge/",
    ],
    env_vars={
        "GOOGLE_GENAI_USE_VERTEXAI": "TRUE",
        "PIZZA_SELLER_AGENT_URL": PIZZA_AGENT_SERVICE_URL,
        "BURGER_SELLER_AGENT_URL": BURGER_AGENT_SERVICE_URL,
    },
)

print(f"Deployed remote app resource: {remote_app.resource_name}")

In [ ]:
print(remote_app.resource_name)
os.environ["AGENT_ENGINE_RESOURCE_NAME"] = remote_app.resource_name

#### Test ADK Agent using curl:

In [ ]:
%%bash
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/${AGENT_ENGINE_RESOURCE_NAME}:streamQuery?alt=sse -d '{
  "class_method": "stream_query",
  "input": {
    "user_id": "user_123",
    "message": "List available burger menu please",
  }
}'


In [ ]:
import requests
import subprocess
import os
import json

# 1. Get the AGENT_ENGINE_RESOURCE_NAME from environment variables
# Example: "projects/your-gcp-project-id/locations/us-central1/agents/your-agent-id/engines/your-engine-id"
agent_engine_resource_name = os.getenv("AGENT_ENGINE_RESOURCE_NAME")
if not agent_engine_resource_name:
    raise ValueError("The 'AGENT_ENGINE_RESOURCE_NAME' environment variable is not set.")

# 2. Construct the full URL
url = f"https://us-central1-aiplatform.googleapis.com/v1/{agent_engine_resource_name}:streamQuery"

# 3. Get the gcloud access token
try:
    token_process = subprocess.run(
        ["gcloud", "auth", "print-access-token"],
        capture_output=True,
        text=True,
        check=True
    )
    access_token = token_process.stdout.strip()
except (subprocess.CalledProcessError, FileNotFoundError) as e:
    print(f"Error getting gcloud access token: {e}")
    print("Please ensure the gcloud CLI is installed, authenticated, and in your system's PATH.")
    exit(1)

# 4. Define the request headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
}

# 5. Define the request payload (data)
payload = {
    "class_method": "stream_query",
    "input": {
        "user_id": "user_123",
        "message": "List available pizza menu please",
    }
}

# 6. Make the POST request with streaming enabled
try:
    with requests.post(
        url,
        headers=headers,
        json=payload, # requests handles JSON serialization
        params={"alt": "sse"}, # Handles the query parameter
        stream=True  # This is key for streaming responses
    ) as response:
        # Check for HTTP errors
        response.raise_for_status()
        
        print("Successfully connected to stream. Waiting for data...")
        # Iterate over the response line by line as data arrives
        stream_chunk = 0
        for line in response.iter_lines():
            if line:
                # Decode bytes to string
                decoded_line = line.decode('utf-8')
                print(f"------------------------[Stream chunk:{str(stream_chunk)}]------------------------")
                print(json.dumps(json.loads(decoded_line), indent=4))
                stream_chunk+=1

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

In [ ]:
import gradio as gr

from typing import List, Dict, Any
from pprint import pformat
from vertexai import agent_engines
import os
from dotenv import load_dotenv

#load_dotenv()

USER_ID = "default_user"

REMOTE_APP = agent_engines.get(os.getenv("AGENT_ENGINE_RESOURCE_NAME"))
SESSION_ID = REMOTE_APP.create_session(user_id=USER_ID)["id"]


async def get_response_from_agent(
    message: str,
    history: List[Dict[str, Any]],
) -> str:
    """Send the message to the backend and get a response.

    Args:
        message: Text content of the message.
        history: List of previous message dictionaries in the conversation.

    Returns:
        Text response from the backend service.
    """
    # try:

    default_response = "No response from agent"

    responses = []

    for event in REMOTE_APP.stream_query(
        user_id=USER_ID,
        session_id=SESSION_ID,
        message=message,
    ):
        parts = event.get("content", {}).get("parts", [])
        if parts:
            for part in parts:
                if part.get("function_call"):
                    #formatted_call = f"```python\n{pformat(part.get('function_call'), indent=2, width=80)}\n```"
                    formatted_call = pformat(part.get('function_call'), indent=2, width=80)
                    responses.append(
                        gr.ChatMessage(
                            role="assistant",
                            content=f"{part.get('function_call').get('name')}:\n{formatted_call}",
                            metadata={"title": "🛠️ Tool Call"},
                        )
                        # {
                        #     "role": "assistant",
                        #     "content": f"{part.get('function_call').get('name')}:\n{formatted_call}",
                        #     "metadata": {"title": "🛠️ Tool Call"},
                        # }
                    )
                elif part.get("function_response"):
                    formatted_response = pformat(part.get('function_response'), indent=2, width=80)

                    responses.append(
                        gr.ChatMessage(
                            role="assistant",
                            content=formatted_response,
                            metadata={"title": "⚡ Tool Response"},
                        )
                        # {
                        #     "role": "assistant",
                        #     "content": formatted_response,
                        #     "metadata": {"title": "⚡ Tool Response"},
                        # }
                    )
                elif part.get("text"):
                    responses.append(
                        gr.ChatMessage(
                            role="assistant",
                            content=part.get("text"),
                        )
                        # {
                        #     "role": "assistant",
                        #     "content": part.get("text"),
                        # }
                    )
                else:
                    formatted_unknown_parts = pformat(part, indent=2, width=80)

                    responses.append(
                        gr.ChatMessage(
                            role="assistant",
                            content=formatted_unknown_parts,
                        )
                        
                        # {
                        #     "role": "assistant",
                        #     "content": formatted_unknown_parts,
                        # }
                    )

    if not responses:
        yield default_response

    yield responses

In [ ]:
%autoawait asyncio
aaa=get_response_from_agent("List available pizza menu please", None)

In [ ]:
# This will wait for the generator to finish and collect all chunks
all_chunks = [chunk async for chunk in get_response_from_agent("List available pizza menu please", None)]
for chunks in all_chunks:
    for chunk in chunks:
        print("--------------------\n")
        print(chunk)